## CSV Data to Delta Tables (Dimension Tables)

#### 1. Configuraciones Previas

In [1]:
# Importar librerias
import os
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from delta.tables import DeltaTable
from pyspark.sql.functions import col, lit, when
from datetime import date

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 3, Finished, Available, Finished)

In [2]:
# Ejecutar sesiones de spark
spark = SparkSession.builder.appName("PerezLLorca").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 4, Finished, Available, Finished)

In [4]:
# Declarar el directorio de ficheros csv de dimensiones en Landing
landing_dim_files = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/'
# Acceso al sistema de ficheros Hadoop
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(landing_dim_files))

# Lista con los nombres de los ficheros CSV
csv_files = [f.getPath().getName() for f in status if f.getPath().getName().endswith(".csv")]

print("Ficheros CSV encontrados:")
print(csv_files)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 6, Finished, Available, Finished)

Ficheros CSV encontrados:
['DimCategoria.csv', 'DimClientes.csv', 'DimCuentasContables.csv', 'DimGeografia.csv', 'DimProducto.csv', 'DimProvincia.csv', 'DimRegionGeografica.csv', 'DimSubcategoria.csv']


In [13]:
## Declarar nombres de tablas
# Tabla Dimensiones
nombre_tabla_cuentas_contables = 'DimCuentasContables'
nombre_tabla_estructura_pl = 'DimEstructuraPL'
nombre_tabla_clientes = 'DimClientes'
nombre_tabla_geografia = 'DimGeografia'
nombre_tabla_provincia = 'DimProvincia'
nombre_tabla_region = 'DimRegionGeografica'
nombre_tabla_producto = 'DimProducto'
nombre_tabla_productocategoria = 'DimCategoria'
nombre_tabla_productosubcategoria = 'DimSubcategoria'

## Declarar directorios
# Directorio ficheros dimension (Lakehouse de Landing)
landing_dim_cuentas_contables = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/DimCuentasContables.csv'
landing_dim_estructura_pl = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/DimEstructuraPL.csv'
landing_dim_clientes = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/DimClientes.csv'
landing_dim_geografia = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/DimGeografia.csv'
landing_dim_provincia = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/DimProvincia.csv'
landing_dim_region = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/DimRegionGeografica.csv'
landing_dim_producto = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/DimProducto.csv'
landing_dim_productocategoria = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/DimCategoria.csv'
landing_dim_productosubcategoria = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/dimensiones/DimSubcategoria.csv'

# Directorio schema dbo
dbo_path = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Tables/dbo/'

# Directorio de las tablas a escribir en schema dbo
# Tablas Delta Dimensiones
path_to_write_dim_cuentas_contables = os.path.join(dbo_path, nombre_tabla_cuentas_contables)
path_to_write_dim_estructura_pl = os.path.join(dbo_path, nombre_tabla_estructura_pl)
path_to_write_dim_clientes = os.path.join(dbo_path, nombre_tabla_clientes)
path_to_write_dim_geografia = os.path.join(dbo_path, nombre_tabla_geografia)
path_to_write_dim_provincia = os.path.join(dbo_path, nombre_tabla_provincia)
path_to_write_dim_region = os.path.join(dbo_path, nombre_tabla_region)
path_to_write_dim_producto = os.path.join(dbo_path, nombre_tabla_producto)
path_to_write_dim_productocategoria = os.path.join(dbo_path, nombre_tabla_productocategoria)
path_to_write_dim_productosubcategoria = os.path.join(dbo_path, nombre_tabla_productosubcategoria)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 15, Finished, Available, Finished)

#### 2. Lectura & Tratamiento de datos

In [9]:
# Carga de datos de la tabla cuentas contables
df_cuentas_contables = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_dim_cuentas_contables)
# Visualizar los datos
#display(df_cuentas_contables)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b6e3daf4-13ae-4b9f-9077-1fae0d37ef2a)

In [15]:
# Carga de datos de la tabla estructura pl
df_estructura_pl = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ";").csv(landing_dim_estructura_pl)
# Visualizar los datos
#display(df_estructura_pl)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4f736805-3cb4-40c4-840e-daca194bcb4c)

In [18]:
# Carga de datos de la tabla clientes
df_clientes = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_dim_clientes)
# Visualizar los datos
#display(df_clientes)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a5b33381-d7a5-466c-8bff-d3a0bbcf4dde)

In [20]:
# Carga de datos de la tabla geografia
df_geografia = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_dim_geografia)
# Visualizar los datos
#display(df_geografia)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 94e1ccf2-2dd9-4761-a8ff-0808899cc034)

In [22]:
# Carga de datos de la tabla geografia
df_provincia = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_dim_provincia)
# Visualizar los datos
#display(df_provincia)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4b2ccd31-b7dd-4e65-aeb2-552b0040eabb)

In [24]:
# Carga de datos de la tabla region
df_region = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_dim_region)
# Visualizar los datos
#display(df_region)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9f82403f-bf57-4109-ab9d-3b35c329f0c3)

In [26]:
# Carga de datos de la tabla producto
df_producto = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_dim_producto)
# Visualizar los datos
#display(df_producto)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fb85afbf-b2cc-4b50-b452-11d025685b8f)

In [28]:
# Carga de datos de la tabla productocategoria
df_productocategoria = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_dim_productocategoria)
# Visualizar los datos
#display(df_productocategoria)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7f03bbe8-d028-44c8-8f1c-061cc419e0e3)

In [30]:
# Carga de datos de la tabla productosubcategoria
df_productosubcategoria = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_dim_productosubcategoria)
# Visualizar los datos
#display(df_productosubcategoria)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 32, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9cba249d-af5f-4ff8-a92b-16e622df9904)

#### 3. Escribir datos en Lakehouse (schema dbo)

In [11]:
# Escribir en Lakehouse contenido df_cuentas_contables
df_cuentas_contables.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_dim_cuentas_contables)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 13, Finished, Available, Finished)

In [16]:
# Escribir en Lakehouse contenido df_estructura_pl
df_estructura_pl.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_dim_estructura_pl)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 18, Finished, Available, Finished)

In [19]:
# Escribir en Lakehouse contenido df_clientes
df_clientes.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_dim_clientes)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 21, Finished, Available, Finished)

In [21]:
# Escribir en Lakehouse contenido df_geografia
df_geografia.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_dim_geografia)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 23, Finished, Available, Finished)

In [23]:
# Escribir en Lakehouse contenido df_provincia
df_provincia.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_dim_provincia)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 25, Finished, Available, Finished)

In [27]:
# Escribir en Lakehouse contenido df_producto
df_producto.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_dim_producto)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 29, Finished, Available, Finished)

In [ ]:
# Escribir en Lakehouse contenido df_region
df_region.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_dim_region)

In [29]:
# Escribir en Lakehouse contenido df_productocategoria
df_productocategoria.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_dim_productocategoria)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 31, Finished, Available, Finished)

In [31]:
# Escribir en Lakehouse contenido df_productosubcategoria
df_productosubcategoria.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_dim_productosubcategoria)

StatementMeta(, 51728fbb-fbf6-48ed-8bdf-c6e0d0b6b54a, 33, Finished, Available, Finished)